In [1]:
from Preprocessor import Preprocessor

#FilterReview
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import GaussianNB 
from sklearn.linear_model import LogisticRegression
from sklearn import metrics 
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import pickle 
import os.path

class SentimentAnalysis:
    def __init__(self):
        self.modelname = 'model/SentimentAnalysis_model.sav'
        self.SA_X_name = 'model/SA_X.sav'
        self.SA_Y_name = 'model/SA_Y.sav'
        
        if os.path.isfile(self.modelname ):
            print('모델을 불러오는중...')
            self.gnb = pickle.load(open(self.modelname , 'rb'))
        else : 
            print('모델이 없습니다!!')
            
    def input_X(self, para):
        pre = Preprocessor()
        print(para)
        pre.setPara(para)
        self.X = pre.transVector()
    def input_Y(self, result):
        self.Y = result

    def transDataLoad(self,tag = True):
        self.X = pickle.load(open(self.SA_X_name , 'rb'))
        if tag == True:
            self.Y = pickle.load(open(self.SA_Y_name , 'rb'))
        
    def transDataSave(self, tag = True):
        pickle.dump(self.X, open(self.SA_X_name, 'wb'))
        if tag == True:
            pickle.dump(self.Y, open(self.SA_Y_name, 'wb'))
            
    def readCSV(self,filename,tag=True, save=True):
        self.data = pd.read_csv(filename,engine='python',encoding='utf-8-sig')
        self.X_content = self.data['sentence'].values
        self.input_X(self.X_content)
        if tag == True:
            self.input_Y(self.data['label'].values)
            if save == True:
                self.transDataSave(tag=True)
        elif save == True:
             self.transDataSave(tag=False)
    def trainModel(self,ts):
        self.X_train,self.X_test,self.y_train,self.y_test=train_test_split(self.X,self.Y,test_size=ts)
        self.gnb = GaussianNB()
        self.gnb = LogisticRegression(random_state=1234)
        print('모델 훈련 중..')
        self.y_pred = self.gnb.fit(self.X_train, self.y_train).predict(self.X_test)
        print('모델 훈련 끝.')
        
        
        # 테스트 socre 확인
        print("Number of right labeled points out of a total %d points : %.4f %%"
      % (len(self.y_test),self.gnb.score(self.X_test, self.y_test)))
        
        pickle.dump(self.gnb, open(self.modelname , 'wb'))
        isSetModel = True
        
        # gnb 55.62 %
    def predict(self):
        self.y_pred = self.gnb.predict(self.X)
        self.df = pd.DataFrame({'ps_label':self.y_pred, 'sentence':self.X_content})
        return self.y_pred
    
    def writePredictionResult(self, filename):
        self.df.to_csv(filename,encoding='utf-8-sig',index=False)
        
    def printAVGScoreResult(self):
        print('예측 평균 점수: ',self.df['ps_label'].mean(),'\n')

C:\Anaconda3\envs\konlpy\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#sa = SentimentAnalysis()
#sa.transDataLoad(tag=True)
#sa.trainModel(ts=0.25)

In [12]:
import glob
import os

#fr.readCSV(tag=False,filename='Data_ReviewFilter/TrainData.csv')
#print('\\n Readed\\n')
#fr.trainModel(ts=0.25)

folderlist = glob.glob('./labeled_data/*')
for fol in folderlist:
    if os.path.isdir(fol):
        print('folder:',fol)
      
        filelist = glob.glob(fol+'/*.csv')
        for file in filelist:
            input_file = file.replace('\\','/')
            input_path = input_file[2:]
            print('input_file: ',input_path)
            print('\\n Readed\\n')
            
            sa = SentimentAnalysis()
            sa.readCSV(tag=False,filename=input_path,save=False)
            sa.predict()
            output_file = 'classifed_'+input_path[18:]
            print('output_file:', output_file)
            sa.writePredictionResult(filename=output_file)
            sa.printAVGScoreResult()

folder: ./labeled_data\2015
input_file:  labeled_data/2015/labeled_오르비_content_수능특강_화학 _2015.01.01. ~ 2015.12.31._split.csv
\n Readed\n
모델을 불러오는중...


C:\Anaconda3\envs\konlpy\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.19.1 when using version 0.19.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


['기출의 고백 추천합니다.' '? 현재 영어공부는 수능 연계 교재만 보고 있습니다.' '쉬는 시간에는 휴대폰을 보지 말아 주세요.'
 ... '유형에만 얽매이지 않는 유연한 학습이 더 중요합니다.'
 '그게 쌓이고 쌓인 뒤 마무리 기간 때쯤에 문득 다시 보면, 어떤 예상 문제가 어떤 페이지에 있었는지 까지 대충 다 파악이 되어 있는 상태로 변한 자신의 모습을 발견할 수 있을 거예요.'
 'ㅠ 제가 못 찾는 건지..']
output_file: classifed_labeled_오르비_content_수능특강_화학 _2015.01.01. ~ 2015.12.31._split.csv
예측 평균 점수:  0.47034277198211627 

folder: ./labeled_data\2016
input_file:  labeled_data/2016/labeled_오르비_content_수능특강_화학 _2016-01-01 ~ 2016-12-31_split.csv
\n Readed\n
모델을 불러오는중...
['( 공부 법 칼럼을 따로 쓰긴 했지만) 요약하자면, 얕은 공부, 요행을 바라는 공부가 아닌, 본질적인 실력자체를 키울 수 있는 공부를 해야 하고, 수학문제는 절대 가려 풀지 않는다.'
 '정상인의 분극상태일 때 세포 내 양전하가 세포 외보다 상대적으로 적어서? - 전위차를 갖게 되는데 세포 외의 칼륨 농도가 감소하면 세포 내의 양전하가 상대적으로 많은 것이고 그럼 - 전 위 ( 음전 위) 의 크기가 감소하여 0에 가깝게 되거나 + 전 위를 가져야 하는 게 아닙니까?'
 '" 요즘은 EBS에서 70% 가 연계되니까 EBS만 잘 봐도 성적 오를 거에요. "14 년도 수능 시즌. 그 당시 만점 자 1% 정책은 폐지되었지만 교육부에서는 항상 EBS를 강조하고 또 강조했었다.'
 ...
 '월에 선 과 탐까지 완성되어 한 손으로 틀린 걸 셀 수 있는 경지에 도달했고, 영어 같은 경우엔 하나의 모의고사에서 막힌 문장표시할 것이 없는 경우까지 가더군요.'
 '화학은 케 미 옵티마? 

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(random_state=1234)
classifier.fit(train_x, train_y)

# 테스트 socre 확인
print( classifier.score(test_x, test_y) )
# 0.63904

# save the model to disk
filename = 'model/finalized_model.sav'
pickle.dump(classifier, open(filename, 'wb'))